#### V1

In [5]:
import cv2
import numpy as np

# Video dosyasını yükleme
video_path = "C:/Users/goksuk/Downloads/cars-traffic.mp4"
cap = cv2.VideoCapture(video_path)

# Arka plan ayırıcıyı oluştur
backSub = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=16, detectShadows=True)

# Araçların izlenmesi için ID oluşturma
vehicle_id = 0
vehicles = {}  # ID'lere göre araçları saklayacağız
vehicle_centroids = []

# Araç sayısı
vehicle_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Çerçeveyi gri tonlamalı hale getirme
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Arka plan ayrıştırıcı uygulama
    fg_mask = backSub.apply(gray)

    # Gürültü temizleme
    fg_mask = cv2.medianBlur(fg_mask, 5)

    # Kontur bulma
    contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Mevcut çerçevede tespit edilen araçların merkez noktalarını depolayacağız
    current_centroids = []

    for contour in contours:
        if cv2.contourArea(contour) > 500:
            x, y, w, h = cv2.boundingRect(contour)
            
            # ROI'nin geçerli olup olmadığını kontrol etme
            if x + w <= frame.shape[1] and y + h <= frame.shape[0]:
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

                # Araç merkez noktasını hesaplama
                centroid = (int(x + w/2), int(y + h/2))
                current_centroids.append(centroid)

                # Yeni araç tespiti ve ID atama
                if not any(np.linalg.norm(np.array(centroid) - np.array(c)) < 20 for c in vehicle_centroids):
                    vehicle_id += 1
                    tracker = cv2.TrackerKCF_create()
                    tracker.init(frame, (x, y, w, h))
                    vehicles[vehicle_id] = tracker
                    vehicle_centroids.append(centroid)

    # Araç takibi ve sayımı
    for vehicle_id, tracker in list(vehicles.items()):
        success, box = tracker.update(frame)
        if success:
            x, y, w, h = map(int, box)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            cv2.putText(frame, f"ID: {vehicle_id}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
            centroid = (int(x + w/2), int(y + h/2))

            # Araç sayımı
            if centroid[1] > frame.shape[0] // 2 and vehicle_id not in vehicle_centroids:
                vehicle_count += 1
                vehicle_centroids.append(vehicle_id)
        else:
            # Takip başarısız olduğunda takipçiyi kaldır
            del vehicles[vehicle_id]

    # Araç sayısını ekrana yazdırma
    cv2.putText(frame, f"Arac Sayisi: {vehicle_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Ekranda gösterme
    cv2.imshow("Arac Tespiti ve Takibi", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


#### V2

In [2]:
import cv2
import numpy as np

# Video dosyasını yükleme
video_path = "C:/Users/goksuk/Downloads/cars-traffic.mp4"
cap = cv2.VideoCapture(video_path)

# Arka plan ayırıcıyı oluştur
backSub = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=16, detectShadows=True)

# Araçların izlenmesi için ID oluşturma
vehicle_id = 0
vehicles = {}  # ID'lere göre araçları saklayacağız
vehicle_centroids = {}
vehicle_ids_counted = set()  # Sayılan araçların ID'lerini saklayacağız

# Araç sayısı
vehicle_count = 0

# Geçiş hattını belirleme (video karesinin ortasında yatay bir çizgi)
line_position = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT) / 2)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Çerçeveyi gri tonlamalı hale getirme
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Arka plan ayrıştırıcı uygulama
    fg_mask = backSub.apply(gray)

    # Gürültü temizleme
    fg_mask = cv2.medianBlur(fg_mask, 5)

    # Kontur bulma
    contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Mevcut çerçevede tespit edilen araçların merkez noktalarını depolayacağız
    current_centroids = []

    for contour in contours:
        if cv2.contourArea(contour) > 500:
            x, y, w, h = cv2.boundingRect(contour)
            
            # ROI'nin geçerli olup olmadığını kontrol etme
            if x + w <= frame.shape[1] and y + h <= frame.shape[0]:
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

                # Araç merkez noktasını hesaplama
                centroid = (int(x + w/2), int(y + h/2))
                current_centroids.append((centroid, (x, y, w, h)))

                # Yeni araç tespiti ve ID atama
                if not any(np.linalg.norm(np.array(centroid) - np.array(c)) < 20 for c, _ in vehicle_centroids.values()):
                    vehicle_id += 1
                    tracker = cv2.TrackerCSRT_create()
                    tracker.init(frame, (x, y, w, h))
                    vehicles[vehicle_id] = tracker
                    vehicle_centroids[vehicle_id] = centroid

    # Araç takibi ve sayımı
    for vehicle_id, tracker in list(vehicles.items()):
        success, box = tracker.update(frame)
        if success:
            x, y, w, h = map(int, box)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            cv2.putText(frame, f"ID: {vehicle_id}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
            centroid = (int(x + w/2), int(y + h/2))

            # Araç merkez noktasını güncelleme
            vehicle_centroids[vehicle_id] = centroid

            # Araç sayımı
            if centroid[1] > line_position and vehicle_id not in vehicle_ids_counted:
                vehicle_count += 1
                vehicle_ids_counted.add(vehicle_id)

        else:
            # Takip başarısız olduğunda takipçiyi kaldır
            del vehicles[vehicle_id]
            del vehicle_centroids[vehicle_id]

    # Geçiş hattını çizme
    cv2.line(frame, (0, line_position), (frame.shape[1], line_position), (0, 0, 255), 2)

    # Araç sayısını ekrana yazdırma
    cv2.putText(frame, f"Arac Sayisi: {vehicle_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Ekranda gösterme
    cv2.imshow("Arac Tespiti ve Takibi", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [4]:
import cv2
import numpy as np

# YOLO model dosyalarının yolları
yolo_config_path = "C:/Users/goksuk/Desktop/Goksu/AI_Engineering/OpenCV/Homework/yolov4.cfg"
yolo_weights_path = "C:/Users/goksuk/Desktop/Goksu/AI_Engineering/OpenCV/Homework/yolov4.weights"
coco_names_path = "C:/Users/goksuk/Desktop/Goksu/AI_Engineering/OpenCV/Homework/coco.names"

# COCO sınıf isimlerini yükleme
with open(coco_names_path, 'r') as f:
    class_names = f.read().strip().split('\n')

# Yalnızca araçları tespit etmek için sınıf ID'leri
vehicle_classes = ["car", "bus", "truck", "motorbike"]
vehicle_class_ids = [class_names.index(cls) for cls in vehicle_classes]

# YOLO modelini yükleme
net = cv2.dnn.readNetFromDarknet(yolo_config_path, yolo_weights_path)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)
layer_names = net.getLayerNames()
output_layer_names = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

# Video dosyasını yükleme
video_path = "/mnt/data/cars-traffic.mp4"
cap = cv2.VideoCapture(video_path)

# Araçların izlenmesi için ID oluşturma
vehicle_id = 0
vehicles = {}  # ID'lere göre araçları saklayacağız
vehicle_centroids = {}
vehicle_ids_counted = set()  # Sayılan araçların ID'lerini saklayacağız

# Araç sayısı
vehicle_count = 0

# Geçiş hattını belirleme (video karesinin ortasında yatay bir çizgi)
line_position = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT) / 2)

def detect_vehicles(frame):
    height, width = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    detections = net.forward(output_layer_names)
    boxes = []
    confidences = []
    class_ids = []

    for output in detections:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if class_id in vehicle_class_ids and confidence > 0.5:
                box = detection[0:4] * np.array([width, height, width, height])
                (centerX, centerY, w, h) = box.astype("int")
                x = int(centerX - (w / 2))
                y = int(centerY - (h / 2))
                boxes.append([x, y, int(w), int(h)])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    if len(indices) > 0:
        boxes = [boxes[i[0]] for i in indices]
        class_ids = [class_ids[i[0]] for i in indices]
    else:
        boxes = []
        class_ids = []

    return boxes, class_ids

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Araç tespiti
    boxes, class_ids = detect_vehicles(frame)

    # Mevcut çerçevede tespit edilen araçların merkez noktalarını depolayacağız
    current_centroids = []

    for box in boxes:
        x, y, w, h = box
        centroid = (int(x + w / 2), int(y + h / 2))
        current_centroids.append((centroid, (x, y, w, h)))
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Yeni araç tespiti ve ID atama
    for centroid, bbox in current_centroids:
        if not any(np.linalg.norm(np.array(centroid) - np.array(c)) < 50 for c, _ in vehicle_centroids.values()):
            vehicle_id += 1
            tracker = cv2.TrackerCSRT_create()
            tracker.init(frame, bbox)
            vehicles[vehicle_id] = tracker
            vehicle_centroids[vehicle_id] = centroid

    # Araç takibi ve sayımı
    for vehicle_id, tracker in list(vehicles.items()):
        success, box = tracker.update(frame)
        if success:
            x, y, w, h = map(int, box)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
            cv2.putText(frame, f"ID: {vehicle_id}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
            centroid = (int(x + w / 2), int(y + h / 2))

            # Araç merkez noktasını güncelleme
            vehicle_centroids[vehicle_id] = centroid

            # Araç sayımı
            if centroid[1] > line_position and vehicle_id not in vehicle_ids_counted:
                vehicle_count += 1
                vehicle_ids_counted.add(vehicle_id)
        else:
            # Takip başarısız olduğunda takipçiyi kaldır
            del vehicles[vehicle_id]
            del vehicle_centroids[vehicle_id]

    # Geçiş hattını çizme
    cv2.line(frame, (0, line_position), (frame.shape[1], line_position), (0, 0, 255), 2)

    # Araç sayısını ekrana yazdırma
    cv2.putText(frame, f"Arac Sayisi: {vehicle_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Ekranda gösterme
    cv2.imshow("Arac Tespiti ve Takibi", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


IndexError: invalid index to scalar variable.